In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# 读取CSV文件
data = pd.read_csv('/content/drive/My Drive/GEE_Export2019/samples_with_features2019.csv')

# 选择特定的特征列
selected_features = ['Peak_val','Base_val','B2','MNDWI','NDVI','RVI','End_val','B12','VH','B3','B4','VV','Start_val','B11','NDWI','b1']

# 提取所选特征列作为已标记数据
labeled_data = data[selected_features]

# 显示提取的特征数据
print(labeled_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      Peak_val  Base_val       B2     MNDWI      NDVI       RVI   End_val  \
0     0.323753  0.183908  0.09365 -0.428763  0.147034  1.344759  0.170342   
1     0.850694  0.631291  0.06800 -0.370383  0.455143  2.670690  0.730591   
2     0.854987  0.829890  0.08440 -0.392233  0.208154  1.525744  0.804793   
3     0.712410  0.387446  0.05960 -0.483099  0.494984  2.960272  0.377349   
4     0.855914  0.613267  0.08220 -0.393714  0.218325  1.558607  0.561573   
...        ...       ...      ...       ...       ...       ...       ...   
5539  0.321329  0.202314  0.09610 -0.291517  0.196155  1.488042  0.178918   
5540  0.494049  0.391442  0.08195 -0.449222  0.329008  1.980663  0.494049   
5541  0.201579  0.116466  0.09910 -0.267839  0.117482  1.266244  0.119717   
5542  0.266835  0.141954  0.10590 -0.289340  0.230523  1.599167  0.143322   
5543  0.236333  0.166507

In [ ]:
data1 = pd.read_csv('/content/drive/My Drive/GEE_Samplesnian/sentinel_samples2019.csv')
selected_features1 = ['Peak_val','Base_val','B2','MNDWI','NDVI','RVI','End_val','B12','VH','B3','B4','VV','Start_val','B11','NDWI',]
unlabeled_data = data1[selected_features1]
print(unlabeled_data)

       Peak_val  Base_val       B2     MNDWI      NDVI       RVI   End_val  \
0      0.887855  0.702429  0.03190 -0.470051  0.495671  2.965665  0.576873   
1      0.485298  0.326968  0.05715 -0.492481  0.354828  2.099948  0.335580   
2      0.831858  0.469792  0.08020 -0.350515  0.445965  2.609881  0.506814   
3      0.732744  0.499026  0.04090 -0.482884  0.623472  4.311688  0.528738   
4      0.415088  0.318123  0.09620 -0.358025  0.242054  1.638710  0.339820   
...         ...       ...      ...       ...       ...       ...       ...   
19944  0.622923  0.506141  0.05940 -0.416639  0.535245  3.303345  0.545896   
19945  0.862129  0.519181  0.03980 -0.531273  0.612430  4.160357  0.451902   
19946  0.846371  0.733523  0.01230 -0.631501  0.658899  4.863372  0.783268   
19947  0.356136  0.249064  0.07815 -0.475786  0.264091  1.717726  0.228723   
19948  0.293656  0.209611  0.08480 -0.464631  0.218014  1.557592  0.155175   

           B12         VH       B3       B4         VV  Start_v

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score, confusion_matrix
import numpy as np
import pandas as pd

# ------------------ 初始划分 ------------------
train_data, test_data1, train_labels, test_labels1 = train_test_split(
    labeled_data.iloc[:, :15], labeled_data.iloc[:, -1], test_size=0.2, random_state=42
)

# 初始化随机森林分类器
model = RandomForestClassifier()
model.fit(train_data, train_labels)

# 预测未标记数据的类别
unlabeled_predictions = model.predict(unlabeled_data.iloc[:, :15])
unlabeled_probability = model.predict_proba(unlabeled_data.iloc[:, :15])

# 设置置信度阈值
threshold = 0.90  # 概率大于 0.9 认为可信

# 合并预测结果与置信度
unlabeled_data['b1'] = unlabeled_predictions
unlabeled_data['probability'] = unlabeled_probability.max(axis=1)

# 选择高置信样本加入标记数据
new_labeled_data = unlabeled_data[unlabeled_data['probability'] > threshold].iloc[:, :-1]
unlabeled_data = unlabeled_data[unlabeled_data['probability'] <= threshold].iloc[:, :-2]

# 合并到标记数据
labeled_data = pd.concat([labeled_data, new_labeled_data], axis=0)

# ------------------ 迭代部分 ------------------
max_iterations = 10          # 最大迭代次数
min_new_samples = 200       # 新增样本数小于该值时停止

best_model = None           # 用于保存上一轮模型
best_iteration = 0          # 保存上一轮的迭代号

for iteration in range(1, max_iterations + 1):
    # 划分训练/测试集
    train_data, test_data, train_labels, test_labels = train_test_split(
        labeled_data.iloc[:, :15], labeled_data.iloc[:, -1], test_size=0.2, random_state=42
    )

    # 训练模型
    model.fit(train_data, train_labels)

    # 保存本轮模型为“上一轮候选模型”
    best_model = model
    best_iteration = iteration

    # 预测未标记数据
    unlabeled_predictions = model.predict(unlabeled_data.iloc[:, :15])
    unlabeled_probability = model.predict_proba(unlabeled_data.iloc[:, :15])

    unlabeled_data['b1'] = unlabeled_predictions
    unlabeled_data['probability'] = unlabeled_probability.max(axis=1)

    # 新增高置信样本
    new_labeled_data = unlabeled_data[unlabeled_data['probability'] > threshold].iloc[:, :-1]
    new_labeled_count = new_labeled_data.shape[0]

    # 更新未标记数据集
    unlabeled_data = unlabeled_data[unlabeled_data['probability'] <= threshold].iloc[:, :-2]
    # 合并新样本
    labeled_data = pd.concat([labeled_data, new_labeled_data], axis=0)

    # 计算当前精度
    train_accuracy = accuracy_score(train_labels, model.predict(train_data))
    test_accuracy = accuracy_score(test_labels, model.predict(test_data))

    print(f"Iteration {iteration}: "
          f"新增样本={new_labeled_count}, "
          f"标记样本总数={labeled_data.shape[0]}, "
          f"剩余未标记样本={unlabeled_data.shape[0]}, "
          f"Train Acc={train_accuracy:.4f}, Test Acc={test_accuracy:.4f}")

    # ---- 停止条件 ----
    if new_labeled_count < min_new_samples:
        print(f"🚫 Iteration {iteration}: 新增样本 {new_labeled_count} 个，停止迭代。")
        print(f"✅ 使用上一次迭代（Iteration {iteration-1}）的模型作为最终模型。")
        break

# ------------------ 最终模型评估（使用上一轮模型） ------------------
final_test_accuracy = accuracy_score(test_labels, best_model.predict(test_data))
print(f"\n✅ Final Test Accuracy (Iteration {best_iteration-1}) = {final_test_accuracy:.4f}")

# 分类报告
print("\nClassification report:\n", classification_report(best_model.predict(test_data), test_labels))

# Kappa系数
kappa = cohen_kappa_score(best_model.predict(test_data), test_labels)
print(f"Kappa值: {kappa:.4f}")

# 混淆矩阵与精度
conf_matrix = confusion_matrix(best_model.predict(test_data), test_labels)
producer_accuracy = np.diag(conf_matrix) / conf_matrix.sum(axis=0)
user_accuracy = np.diag(conf_matrix) / conf_matrix.sum(axis=1)

print("\nProducer's Accuracy:")
for i, accuracy in enumerate(producer_accuracy):
    print(f"Class {i}: {accuracy:.4f}")

print("\nUser's Accuracy:")
for i, accuracy in enumerate(user_accuracy):
    print(f"Class {i}: {accuracy:.4f}")


/tmp/ipython-input-1539217636.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_data['b1'] = unlabeled_predictions


Iteration 1: 新增样本=1875, 标记样本总数=10824, 剩余未标记样本=14669, Train Acc=1.0000, Test Acc=0.8648
Iteration 2: 新增样本=989, 标记样本总数=11813, 剩余未标记样本=13680, Train Acc=1.0000, Test Acc=0.8868
Iteration 3: 新增样本=679, 标记样本总数=12492, 剩余未标记样本=13001, Train Acc=1.0000, Test Acc=0.8950
Iteration 4: 新增样本=434, 标记样本总数=12926, 剩余未标记样本=12567, Train Acc=1.0000, Test Acc=0.9028
Iteration 5: 新增样本=321, 标记样本总数=13247, 剩余未标记样本=12246, Train Acc=1.0000, Test Acc=0.9006
Iteration 6: 新增样本=265, 标记样本总数=13512, 剩余未标记样本=11981, Train Acc=1.0000, Test Acc=0.9094
Iteration 7: 新增样本=214, 标记样本总数=13726, 剩余未标记样本=11767, Train Acc=1.0000, Test Acc=0.9123
Iteration 8: 新增样本=145, 标记样本总数=13871, 剩余未标记样本=11622, Train Acc=1.0000, Test Acc=0.9097
Iteration 9: 新增样本=138, 标记样本总数=14009, 剩余未标记样本=11484, Train Acc=1.0000, Test Acc=0.9067
Iteration 10: 新增样本=128, 标记样本总数=14137, 剩余未标记样本=11356, Train Acc=1.0000, Test Acc=0.9115

✅ Final Test Accuracy (Iteration 9) = 0.9115

Classification report:
               precision    recall  f1-score   support

           1

In [ ]:
# !pip install geemap scikit-learn
import ee
import geemap
import pandas as pd

from geemap import ml
from sklearn import ensemble
ee.Authenticate()
ee.Initialize(project='liutaogee')
print(labeled_data)

       Peak_val  Base_val       B2     MNDWI      NDVI       RVI   End_val  \
0      0.378739  0.235150  0.10260 -0.334235  0.166667  1.400000  0.206807   
1      0.828646  0.607501  0.06775 -0.403000  0.442180  2.585387  0.617815   
2      0.750200  0.625344  0.12455 -0.287513  0.257726  1.694424  0.750200   
3      0.811871  0.707697  0.06440 -0.432787  0.432575  2.524697  0.650082   
4      0.743210  0.476999  0.11295 -0.204977  0.169392  1.407873  0.538413   
...         ...       ...      ...       ...       ...       ...       ...   
19663  0.371450  0.287698  0.05650 -0.562615  0.301610  1.863728  0.358014   
19664  0.792261  0.648376  0.05370 -0.419064  0.481977  2.860835  0.504492   
19780  0.787649  0.440819  0.09805 -0.401110  0.220779  1.566667  0.394261   
19900  0.362747  0.272743  0.15260 -0.293835  0.167883  1.403509  0.327400   
19971  0.811295  0.517161  0.06400 -0.386541  0.518957  3.157635  0.608858   

          B12         VH       B3       B4         VV  Start_va

In [ ]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
# create a classifier and fit
n_trees = 100
X=labeled_data.iloc[:, :15]
y=labeled_data.iloc[:, -1]
rf = ensemble.RandomForestClassifier(n_trees).fit(X, y)
trees = ml.rf_to_strings(rf, selected_features1)
# print the first tree to see the result
#print(trees[0])
#print(trees[1])
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [ ]:
# -*- coding: utf-8 -*-
"""
Sentinel-2 NDVI Phenological Features Extraction (1 year)
生长期内特征 & 分段积分（SOS->Peak, Peak->EOS）
"""


# --------------------- 1. 参数设置 ---------------------
year = '2019'
startDate = ee.Date(f'{year}-01-01')
endDate = ee.Date(f'{year}-12-31')
roi = ee.FeatureCollection('projects/liutaogee/assets/WeiheRiverBasinpoly').geometry().bounds()  # 替换为研究区

# --------------------- 2. 加载 Sentinel-2 数据并计算 NDVI ---------------------
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(roi)
      .filterDate(startDate, endDate)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      .map(lambda img: img.normalizedDifference(['B8', 'B4'])
           .rename('NDVI')
           .copyProperties(img, ['system:time_start'])))

# --------------------- 3. 逐月合成 NDVI ---------------------
months = ee.List.sequence(1, 12)

def monthly_ndvi(m):
    m = ee.Number(m)
    start = ee.Date(f'{year}-01-01').advance(m.subtract(1), 'month')
    end = start.advance(1, 'month')
    monthlyCol = s2.filterDate(start, end)
    monthlyNDVI = (monthlyCol.mean()
                   .set('month', m)
                   .set('system:time_start', start.millis()))
    return monthlyNDVI

ndviCol = ee.ImageCollection(months.map(monthly_ndvi))
ndviList = ndviCol.toList(ndviCol.size())

# --------------------- 4. 计算阈值并识别生长期（NDVI > 0.5 * NDVImax） ---------------------
ndviMax = ndviCol.max().rename('NDVImax')
ndviThr = ndviMax.multiply(0.5).rename('NDVIthr')

def month_mask(m):
    m = ee.Number(m)
    img = ee.Image(ndviList.get(m.subtract(1)))
    mask = img.gt(ndviThr)
    return ee.Image.constant(m).updateMask(mask).toFloat()

monthMaskCol = ee.ImageCollection.fromImages(months.map(month_mask))
sosImg = monthMaskCol.min().unmask(0).rename('SOS_time')
eosImg = monthMaskCol.max().unmask(0).rename('EOS_time')

# --------------------- 5. 构建生长期 NDVI 影像 ---------------------
def growing_season(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    cond = ee.Image.constant(m).gte(sosImg).And(ee.Image.constant(m).lte(eosImg))
    return nd.updateMask(cond).set('month', m)

growingSeasonCol = ee.ImageCollection.fromImages(months.map(growing_season))

# --------------------- 6. 生长期内基础特征 ---------------------
def start_val(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    return nd.updateMask(sosImg.eq(m))

Start_val = ee.ImageCollection.fromImages(months.map(start_val)).mosaic().unmask(0).rename('Start_val')

def end_val(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    return nd.updateMask(eosImg.eq(m))

End_val = ee.ImageCollection.fromImages(months.map(end_val)).mosaic().unmask(0).rename('End_val')
Base_val = Start_val.add(End_val).divide(2).rename('Base_val')
Peak_val = growingSeasonCol.max().rename('Peak_val')
Ampl = Peak_val.subtract(Base_val).rename('Ampl')

# --------------------- 7. 生长期内差分与生长速率 ---------------------
gsList = ee.List(months.map(lambda m:
    ee.Image(ndviList.get(ee.Number(m).subtract(1)))
    .updateMask(ee.Image.constant(m).gte(sosImg).And(ee.Image.constant(m).lte(eosImg)))
))

def diff_img(i):
    i = ee.Number(i)
    before = ee.Image(gsList.get(i.subtract(1)))
    after = ee.Image(gsList.get(i))
    return after.subtract(before).rename('diff').set('index', i)

diffCol = ee.ImageCollection.fromImages(ee.List.sequence(1, 11).map(diff_img))
L_deriv = diffCol.max().rename('L_deriv')
R_deriv = diffCol.min().multiply(-1).rename('R_deriv')

# --------------------- 8. Peak_month ---------------------
tol = 1e-6
def peak_month_img(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    cond = nd.subtract(Peak_val).abs().lte(tol)
    return ee.Image.constant(m).updateMask(cond).toFloat()

peakMonthCol = ee.ImageCollection.fromImages(months.map(peak_month_img))
Peak_month = peakMonthCol.min().unmask(0).rename('Peak_month')

# --------------------- 9. 分段积分 ---------------------
def pre_peak(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    cond = ee.Image.constant(m).gte(sosImg).And(ee.Image.constant(m).lte(Peak_month))
    return nd.updateMask(cond)

Integral_prePeak = ee.ImageCollection.fromImages(months.map(pre_peak)).sum().unmask(0).rename('Integral_prePeak')

def post_peak(m):
    m = ee.Number(m)
    nd = ee.Image(ndviList.get(m.subtract(1)))
    cond = ee.Image.constant(m).gte(Peak_month).And(ee.Image.constant(m).lte(eosImg))
    return nd.updateMask(cond)

Integral_postPeak = ee.ImageCollection.fromImages(months.map(post_peak)).sum().unmask(0).rename('Integral_postPeak')
L_integral = Integral_prePeak.add(Integral_postPeak).rename('L_integral')

# --------------------- 10. 合并所有物候特征 ---------------------
phenology = (ndviMax.addBands([
    ndviThr, sosImg, Peak_month, eosImg,
    Start_val, End_val, Base_val, Peak_val, Ampl,
    Integral_prePeak, Integral_postPeak, L_integral,
    L_deriv, R_deriv
]).clip(roi).toFloat())

# --------------------- 11. Sentinel-2 光谱指数 ---------------------
s21 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
       .filterBounds(roi)
       .filterDate(startDate, endDate)
       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
       .map(lambda img: img.divide(10000)
            .select(['B2','B3','B4','B8','B11'])
            .copyProperties(img, ['system:time_start'])))

s2mean = s21.median()
ndvi = s2mean.normalizedDifference(['B8','B4']).rename('NDVI')
ndwi = s2mean.normalizedDifference(['B3','B8']).rename('NDWI')
gcvi = s2mean.expression('B8 / B3 - 1', {'B8': s2mean.select('B8'), 'B3': s2mean.select('B3')}).rename('GCVI')
evi = s2mean.expression('2.5 * (B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1)',
                        {'B8': s2mean.select('B8'), 'B4': s2mean.select('B4'), 'B2': s2mean.select('B2')}).rename('EVI')
savi = s2mean.expression('((B8 - B4) / (B8 + B4 + 0.5)) * (1.5)',
                         {'B8': s2mean.select('B8'), 'B4': s2mean.select('B4')}).rename('SAVI')
rvi = s2mean.expression('B8 / B4', {'B8': s2mean.select('B8'), 'B4': s2mean.select('B4')}).rename('RVI')
wi = s2mean.expression('(B3 + 2.5 * B11) / (1.5 * B8 + 2 * B4 + 0.5 * B11)',
                       {'B3': s2mean.select('B3'), 'B8': s2mean.select('B8'),
                        'B4': s2mean.select('B4'), 'B11': s2mean.select('B11')}).rename('WI')
msi = s2mean.expression('B11 / B8', {'B11': s2mean.select('B11'), 'B8': s2mean.select('B8')}).rename('MSI')
ndbi = s2mean.normalizedDifference(['B11','B8']).rename('NDBI')
ndmi = s2mean.normalizedDifference(['B8','B11']).rename('NDMI')
mndwi = s2mean.normalizedDifference(['B3','B11']).rename('MNDWI')
dvi = s2mean.expression('B8 - B4', {'B8': s2mean.select('B8'), 'B4': s2mean.select('B4')}).rename('DVI')

spectral = ndvi.addBands([ndwi, gcvi, evi, savi, rvi, wi, msi, ndbi, ndmi, mndwi, dvi])

# --------------------- 12. 纹理特征 ---------------------
gray = s2mean.expression('0.3 * B8 + 0.59 * B4 + 0.11 * B3',
                         {'B8': s2mean.select('B8'), 'B4': s2mean.select('B4'), 'B3': s2mean.select('B3')}).rename('Gray')
glcm = gray.multiply(100).toInt().glcmTexture(size=3)
texture = glcm.select(['Gray_contrast', 'Gray_asm', 'Gray_corr', 'Gray_var',
                       'Gray_idm', 'Gray_diss', 'Gray_sent', 'Gray_ent'])

# --------------------- 13. Sentinel-2 原始波段 ---------------------
s2_bands = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(roi)
            .filterDate(startDate, endDate)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            .map(lambda img: img.select(
                ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']).divide(10000)
                 .copyProperties(img, ['system:time_start'])))
s2_year = s2_bands.median().clip(roi)

# --------------------- 14. Sentinel-1 数据 ---------------------
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(roi)
             .filterDate(startDate, endDate)
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
             .filter(ee.Filter.eq('instrumentMode', 'IW')))

vvVhIwAsc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vvVhIwDesc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

vvIwAscDescMean = vvVhIwAsc.merge(vvVhIwDesc).select('VV').mean().clip(roi)
vhIwAscDescMean = vvVhIwAsc.merge(vvVhIwDesc).select('VH').mean().clip(roi)

# --------------------- 15. 合并全部特征 ---------------------
allFeatures = (phenology
               .addBands(spectral)
               .addBands(texture)
               .addBands(s2_year)
               .addBands(vvIwAscDescMean)
               .addBands(vhIwAscDescMean)
               .clip(roi)
               .toFloat())

print(allFeatures.getInfo())
print(allFeatures.bandNames().getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVImax', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 5], 'origin': [103, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVIthr', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 5], 'origin': [103, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SOS_time', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 5], 'origin': [103, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'Peak_month', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 5], 'origin': [103, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'EOS_time', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 5], 'origin': [103, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'Start_val', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dim

In [ ]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()

Map = geemap.Map()
# 取原始波段
imageBands = allFeatures.select([
    'Peak_val','Base_val','B2','MNDWI','NDVI','RVI','End_val',
    'B12','VH','B3','B4','VV','Start_val','B11','NDWI'
])
feature_names = ['Peak_val','Base_val','B2','MNDWI','NDVI','RVI','End_val','B12','VH','B3','B4','VV','Start_val','B11','NDWI']


# --------------------- 1. SNIC 分割 ---------------------
def runSNIC(image, seed_pitch, size):
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(seed_pitch)
    snic = ee.Algorithms.Image.Segmentation.SNIC(
        image=image,
        size=size,
        compactness=5,
        connectivity=8,
        neighborhoodSize=2 * size ,
        seeds=seeds
    )
    return snic.select('clusters').rename(f'clusters_{seed_pitch}')

seg60 = runSNIC(imageBands, 60, 128)


# --------------------- 2. 计算对象特征 ---------------------
# 将原始波段添加到 SNIC 对象中
seg60_withBands = seg60.addBands(feature_names)
obj_features = seg60_withBands.reduceConnectedComponents(
    reducer=ee.Reducer.mean(),
    labelBand='clusters_60'
)


# ---------------- 对象分类 ----------------
classified = obj_features.select(feature_names).classify(classifier)






In [ ]:


task = ee.batch.Export.image.toDrive(
    image=classified,              # 要导出的影像
    description='classified_to_drive',   # 任务名（Drive任务描述）
    folder='GEE_exports0000',          # Drive 文件夹名称（提前创建或自动生成）
    fileNamePrefix='classified_2019',    # 文件名前缀
    region=roi,                    # 导出区域
    scale=10,                     # 分辨率（Sentinel 建议用 10）
    crs='EPSG:4326',               # 坐标系
    maxPixels=1e13                 # 允许最大像素数
)

task.start()
print('✅ 任务已启动，请在 https://code.earthengine.google.com/tasks 查看进度')

✅ 任务已启动，请在 https://code.earthengine.google.com/tasks 查看进度
